In [2]:
from vina import Vina
import subprocess
import rdkit.Chem as Chem
from rdkit.Chem import AllChem

In [3]:
def sdf2centroid(sdf_file): # Get the centroid coordinate of ligand.
    supp = Chem.SDMolSupplier(sdf_file, sanitize=False)
    lig_xyz = supp[0].GetConformer().GetPositions()
    centroid_x = '%.4f'% lig_xyz[:,0].mean()
    centroid_y = '%.4f'% lig_xyz[:,1].mean()
    centroid_z = '%.4f'% lig_xyz[:,2].mean()
    return centroid_x, centroid_y, centroid_z

In [4]:
centroid_x, centroid_y, centroid_z = sdf2centroid(f'HDAC6_8bjk_ligand_A.sdf')
lig_pdbqt = 'HDAC6_8bjk_ligand_A.pdbqt'
rectz_pdbqt = 'HDAC6_8bjk_rec_A_tz.pdbqt'

command = f'''pythonsh ../../Dock/autodock_scripts/prepare_gpf4zn.py \
                -l {lig_pdbqt} \
                -r {rectz_pdbqt} \
                -o protein_tz.gpf \
                -p npts=20,20,20 \
                -p gridcenter={centroid_x},{centroid_y},{centroid_z}  \
                –p parameter_file=AD4Zn.dat'''

In [10]:
proc = subprocess.Popen(
                command, 
                shell=True, 
                stdin=subprocess.PIPE, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)

print(proc.communicate()[1].decode('utf-8'))

### Generate the docking grid

**AD4Zn.dat** must be downloaded in the working directory before running the code below.  
It can be download at [here](https://github.com/ccsb-scripps/AutoDock-Vina/blob/develop/data/AD4Zn.dat).

In [12]:
command2 = '''autogrid4 -p protein_tz.gpf -l protein_tz.glg'''
proc = subprocess.Popen(
                command2, 
                shell=True, 
                stdin=subprocess.PIPE, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)

print(proc.communicate()[1].decode('utf-8'))

In [3]:
from openbabel import pybel
from meeko import MoleculePreparation

class PrepLig(object):
    def __init__(self, input_mol, mol_format):
        if mol_format == 'smi':
            self.ob_mol = pybel.readstring('smi', input_mol)
        elif mol_format == 'sdf': 
            self.ob_mol = next(pybel.readfile(mol_format, input_mol))
        else:
            raise ValueError(f'mol_format {mol_format} not supported')
        
    def addH(self, polaronly=False, correctforph=True, PH=7): 
        self.ob_mol.OBMol.AddHydrogens(polaronly, correctforph, PH)
        self.ob_mol.write('sdf', filename='tmp_h.sdf')

    def gen_conf(self):
        sdf_block = self.ob_mol.write('sdf')
        rdkit_mol = Chem.MolFromMolBlock(sdf_block, removeHs=False)
        AllChem.EmbedMolecule(rdkit_mol, Chem.rdDistGeom.ETKDGv3())
        self.ob_mol = pybel.readstring('sdf', Chem.MolToMolBlock(rdkit_mol))
        self.ob_mol.write('sdf', filename='conf_h.sdf')


    def get_pdbqt(self, lig_pdbqt=None):
        preparator = MoleculePreparation()
        preparator.prepare(self.ob_mol.OBMol)
        if lig_pdbqt is not None: 
            preparator.write_pdbqt_file(lig_pdbqt)
            return 
        else: 
            return preparator.write_pdbqt_string()

In [ ]:
lig = PrepLig('5wgl_S_AH4.sdf', '.sdf')
lig.addH()
lig.get_pdbqt('5wgl_S_AH4.pdbqt')